In [1]:
import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': 'iTfSE_YVE6zqnxjd1oK0E37R2aVYY4dFhXFFCljE1AcJ',
    'service_id': 'iam-ServiceId-d4b06e46-293a-4417-b76c-2f16076a9353',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token'}

configuration_name = 'os_b0f1407510994fd1b793b85137baafb8_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face issues with the DataFrame layout.
# Please read the documentation of 'SparkSession.read()' to learn more about the possibilities to adjust the data loading.
# PySpark documentation: http://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.json


df = spark.read.parquet(cos.url('hmp.parquet', 'courseraml-donotdelete-pr-qve0ttzezdeodc'))

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20191011075947-0000
KERNEL_ID = a5473c05-94f8-40c9-8e6e-aed2ff48e7a2


In [2]:
df.createOrReplaceTempView('df')
df_energy = spark.sql("""
select sqrt(sum(x*x)+sum(y*y)+sum(z*z)) as label, class from df group by class
""")      
df_energy.createOrReplaceTempView('df_energy')          

In [3]:
df_join = spark.sql('select * from df inner join df_energy on df.class=df_energy.class')

In [4]:
df_join.show()

+---+---+---+--------------------+------------+------------------+------------+
|  x|  y|  z|              source|       class|             label|       class|
+---+---+---+--------------------+------------+------------------+------------+
| 30| 36| 52|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 30| 36| 32|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 32| 33| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 30| 37| 50|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 31| 37| 50|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|
| 29| 38| 50|Accelerometer-201...|Climb_

In [5]:

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer


vectorAssembler = VectorAssembler(inputCols=["x","y","z"],
                                  outputCol="features")
normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)



In [6]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)



In [7]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, normalizer,lr])


In [8]:
model = pipeline.fit(df_join)


In [9]:
prediction = model.transform(df_join)


In [10]:
prediction.show()

+---+---+---+--------------------+------------+------------------+------------+----------------+--------------------+------------------+
|  x|  y|  z|              source|       class|             label|       class|        features|       features_norm|        prediction|
+---+---+---+--------------------+------------+------------------+------------+----------------+--------------------+------------------+
| 30| 36| 52|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|[30.0,36.0,52.0]|[0.25423728813559...|12378.337524478058|
| 30| 36| 32|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|[30.0,36.0,32.0]|[0.30612244897959...|10949.223755865942|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|[32.0,30.0,36.0]|[0.32653061224489...|11232.049623881609|
| 32| 30| 36|Accelerometer-201...|Climb_stairs|11082.626493751379|Climb_stairs|[32.0,30.0,36.0]|[0.32653061224489...|11232.049623881609|
| 32| 30| 36|Accelerometer-201...|Climb_s

In [11]:
model.stages[2].summary.r2


0.0912330479847846